### Explore Prior classification using different tree based models using only draft vector

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import SVC
import time

In [ ]:
# Make a model using draft only and try different models with it
# extract more features from the database

In [2]:
train_df = pd.read_parquet("../datasets/interim/only_draft_train_set.parquet")
test_df = pd.read_parquet("../datasets/interim/only_draft_test_set.parquet")

In [3]:
train_df.shape

(45000, 227)

#### Prepare the train and test sets

In [4]:
X_train  = train_df.drop(['start_time', 'match_id', 'radiant_win'], axis=1)
y_train = train_df['radiant_win']

X_test  = test_df.drop(['start_time', 'match_id', 'radiant_win'], axis=1)
y_test = test_df['radiant_win']

In [5]:
train_df

,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,hero_8,hero_9,hero_10,...,hero_218,hero_219,hero_220,hero_221,hero_222,hero_223,hero_224,match_id,radiant_win,start_time
9803,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1977637967,1,1449192304
14319,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,2114881692,0,1454142692
33958,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,2346900867,0,1462633835
6172,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1806281138,0,1442725940
32399,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,2335543842,1,1462193600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49144,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,2436256171,1,1465921269
11843,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2050560006,1,1451791977
35221,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2357461259,1,1463051142
43753,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2409312248,0,1464949913


In [6]:

# n_estimators = 10
# start = time.time()
# bagging_clf = BaggingClassifier(
#     SVC(kernel='linear', probability=True, C=0.7),
#     max_samples=1.0 / n_estimators,
#     n_estimators=n_estimators
# )
# bagging_clf.fit(X_train, y_train)
# end = time.time()
# print("Bagging SVC", end - start, bagging_clf.score(X_test, y_test))
# # proba = bagging_clf.predict_proba(X_test)

# # svc = SVC(kernel='linear')
# # svc.fit(X_train, y_train)

logistic_regression = LogisticRegression(max_iter=int(1e6), penalty='none')
logistic_regression.fit(X_train, y_train)

LogisticRegression(max_iter=1000000, penalty='none')

In [7]:
logistic_regression.score(X=X_test, y=y_test)

0.5888

In [17]:
from sklearn.ensemble import RandomForestClassifier


0.528

In [ ]:
RandomForestClassifier()

In [21]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

random_grid = {
    'n_estimators': np.linspace(start = 200, stop = 1000, num = 10, dtype=int),
    'max_features': ['auto', 'sqrt'],
    'max_depth': np.linspace(10, 110, num = 11, dtype=int).append(None),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


In [22]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time= 1.1min
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time= 1.1min
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time= 1.1min


/usr/local/Caskroom/miniconda/base/envs/dota2_predictions/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/Caskroom/miniconda/base/envs/dota2_predictions/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time= 1.5min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time= 1.6min


/usr/local/Caskroom/miniconda/base/envs/dota2_predictions/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time= 2.0min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time= 2.0min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=2000; total time= 2.0min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time= 1.6min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time= 1.7min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time= 1.7min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time= 1.8min
[CV] END bootstrap=False, max_depth=30, 

...

/usr/local/Caskroom/miniconda/base/envs/dota2_predictions/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time= 2.7min


/usr/local/Caskroom/miniconda/base/envs/dota2_predictions/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1800; total time= 8.6min
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1800; total time= 8.7min
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1800; total time= 8.6min
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time= 2.7min
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time= 2.7min
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1800; total time=11.1min


KeyboardInterrupt: 